In [1]:

class Processor(object): #it needs to be modified
    def __init__(self, dbPathOrUrl):
        self.dbPathOrUrl = dbPathOrUrl
        self.dbPathOrUrl = ""
        
    def setDbPathOrUrl(self, new_path):
        self.dbPathOrUrl= new_path
        if new_path:
            return True 
        else: 
            return False
    def getDbPathOrUrl(self):
        return self.dbPathOrUrl


class AnnotationProcessor(Processor): 
    def __init__(self, dbPathOrUrl):
        super().__init__(dbPathOrUrl)

    def uploadData(self, path):
        pass 

class MetadataProcessor(Processor):
    def __init__(self, dbPathorUrl):
        super().__init__(dbPathorUrl)
    def uploadData(self, path1):
        pass 
        

The first step is to model tables that can be uploaded into the database. I did this following the guidelines available in the 4th handson lesson. First, I created tables for all the entities that do not have subclasses, then I worked on the relations between classes -> keep in mind that subclasses inherit attributes and relations of the super-class. For each table, I added an additional column that contains the Internal identifier (primary key) used to identify uniquely the entities. The value of the columns related to relations points to an internal identifier of another table (I did this in the "ImageId" column in the image_ids table). In case of multivalued attributes, the guidelines suggest creating a proxy table that assigns an internal identifier for each of the attributes -> I created a proxy table for "creators". The proxy table's internal identifier is shared in the "creator_id" column of "Entity with metadata" table (here is considered to be as a foreign key).

In [1]:
from sqlite3 import connect
from pandas import read_csv, Series

with connect("relational.db") as con: 
        con.commit()
        
path = read_csv("data/annotations.csv", keep_default_na=False, dtype={"id":"string", "body":"string", "target":"string", "motivation":"string"})

image_ids = path[["body"]]
image_internal_id = []
for idx, row in image_ids.iterrows():
    image_internal_id.append("image-" + str(idx))

image_ids.insert(0, "imageId", Series(image_internal_id, dtype="string"))

annotations_ids = path[["id", "target", "motivation"]]
annotations_ids = annotations_ids.rename(columns={"id": "annotation"})
annotations_internal_id = []
for idx, row in annotations_ids.iterrows():
            annotations_internal_id.append("annotations-" + str(idx))

annotations_ids.insert(0, "annotationsId", Series(annotations_internal_id, dtype="string"))
annotations_ids = annotations_ids.join(image_ids["imageId"]) 
annotations_ids
                    



,annotationsId,annotation,target,motivation,imageId
0,annotations-0,https://dl.ficlit.unibo.it/iiif/2/28429/annota...,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p1,painting,image-0
1,annotations-1,https://dl.ficlit.unibo.it/iiif/2/28429/annota...,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p2,painting,image-1
2,annotations-2,https://dl.ficlit.unibo.it/iiif/2/28429/annota...,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p3,painting,image-2
3,annotations-3,https://dl.ficlit.unibo.it/iiif/2/28429/annota...,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p4,painting,image-3
4,annotations-4,https://dl.ficlit.unibo.it/iiif/2/28429/annota...,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p5,painting,image-4
...,...,...,...,...,...
266,annotations-266,https://dl.ficlit.unibo.it/iiif/2/19425/annota...,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,painting,image-266
267,annotations-267,https://dl.ficlit.unibo.it/iiif/2/19425/annota...,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,painting,image-267
268,annotations-268,https://dl.ficlit.unibo.it/iiif/2/19425/annota...,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,painting,image-268
269,annotations-269,https://dl.ficlit.unibo.it/iiif/2/19425/annota...,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,painting,image-269


### Entity with metadata processor

I had to separate the multivalued attributes that in the CSV file were represented in the proxy table as a unique string separated by ";" ("Doe, John; Doe, Jane"). The multivalued attributes need to be separate because each attribute needs to be associated to a unique identifier (i.e. John Doe could be the author of other books, for this reason he needs to be separated from Jane Doe and be considered as a single attribute). 
- I tried to do this by accessing to the "creator" column of the dataframe and then tranforming the Series into a list. 
- I iterated each element in the list and if there is a multivalued attribute (;) 
- I used .split to divide the two values: the method divides a string into substrings and puts them into a list.  
- Now instead of a string we have sublists of two or more items inside the main list. 
- I iterated again the list to find the sublists, each of their items is inserted into the main list and deleted from the sublist. 
- Finally, I removed the empty sublist. 
- I converted the list into a Dataframe.

I think that this is a very intricate method to separate multivalued attributes, I'll try to work on better solutions. 

In [3]:
from sqlite3 import connect
from pandas import read_csv, Series, DataFrame

path1 = read_csv("data/metadata.csv", keep_default_na=False, dtype={"id":"string", "creator":"string", "title":"string"})
proxy = path1[["creator"]]
proxy1 = proxy["creator"]
list1= proxy1.tolist()
for idx, i in enumerate(list1):
    if ";" in i:
        x = i.split(";") 
        list1.remove(i)
        list1.insert(idx,x)
for idx, z in enumerate(list1):
    if type(z) ==list:
        for k in z:
            list1.insert(idx,k)
            z.remove(k)
            if len(z) == 0:
                list1.remove(z)

proxy_dataframe = DataFrame({"creator": list1})

proxy_id = []
for idx, row in proxy_dataframe.iterrows():
    proxy_id.append("creator-" + str(idx))
proxy_dataframe.insert(0, "creator_id", Series(proxy_id, dtype="string"))
proxy_dataframe

,creator_id,creator
0,creator-0,"Doe, John"
1,creator-1,"Doe, Jane"
2,creator-2,"Alighieri, Dante"
3,creator-3,
4,creator-4,
...,...,...
271,creator-271,
272,creator-272,
273,creator-273,
274,creator-274,


The main goal in this step was to represent correctly in the "Entity with metadata" table the association between id, title and creator. The creator in this table is represented as a foreign key that points directly to the proxy one. 
The main problem was trying to associate single creators to their work, even though before they were considered co-creators(represented as a single string containing 2 names).
 
By looking at the RELATIONAL QUERY PROCESSOR, in particular the method "getEntitiesWithCreator", I realized that we should access to all metadata related to the entities by considering as input ONE OF THE CREATORS.
For this reason, I decided to create an identical row below each book that has been written by more than one author. 
After that, I joined "creators_id" into this table. 

i.e. I cloned the row in which John Doe and Jane Doe are considered authors. I deleted the "creator" column and then I reassigned it to the "creator_id". In this way both John Doe and Jane Doe are considered singularly as creators of "Dante Alighieri: Opere". As a consequence, I should be able to access the item anyways, if I search for it considering only one of the creators. 

I think that this method works only with the data provided, or with 2 co-authors. I need to add a condition that makes it possible to clone a row more than once if the coauthors are >2. 

In [12]:
import pandas as pd
path1 = read_csv("data/metadata.csv", keep_default_na=False, dtype={"id":"string", "creator":"string", "title":"string"})
metadata = path1[["id", "title","creator"]]
metadata_id = []
for idx, row in metadata.iterrows():
            metadata_id.append("metadata-" + str(idx))

for index, row in metadata.iterrows():
        for item_idx, item in row.items():
                if item_idx =="creator":
                        if ";" in item:
                                row_to_copy = metadata.loc[index:index]  
                                metadata = pd.concat([metadata.loc[:index], row_to_copy, metadata.loc[index+1:]]).reset_index(drop=True)

metadata = metadata.drop(["creator"], axis = 1)          
metadata = metadata.join(proxy_dataframe["creator_id"])   

metadata_id = []
for idx, row in metadata.iterrows():
    metadata_id.append("metadata-" + str(idx))
metadata.insert(0, "metadata_internal_id", Series(metadata_id, dtype="string"))

metadata


,metadata_internal_id,id,title,creator_id
0,metadata-0,https://dl.ficlit.unibo.it/iiif/28429/collection,Dante Alighieri: Opere,creator-0
1,metadata-1,https://dl.ficlit.unibo.it/iiif/28429/collection,Dante Alighieri: Opere,creator-1
2,metadata-2,https://dl.ficlit.unibo.it/iiif/2/28429/manifest,Il Canzoniere,creator-2
3,metadata-3,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p1,,creator-3
4,metadata-4,https://dl.ficlit.unibo.it/iiif/2/28429/canvas/p2,,creator-4
...,...,...,...,...
271,metadata-271,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,creator-271
272,metadata-272,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,creator-272
273,metadata-273,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,creator-273
274,metadata-274,https://dl.ficlit.unibo.it/iiif/2/19425/canvas...,,creator-274


In [61]:
with connect("relational.db") as con:   
    annotations_ids.to_sql("annotationsId", con, if_exists="replace", index=False)
    image_ids.to_sql("imageId", con, if_exists="replace", index=False)  
    metadata.to_sql("metadataId", con, if_exists="replace", index=False)
    proxy_dataframe.to_sql("creatorsId", con, if_exists="replace", index=False)